# Import + Preprocessing

In [15]:
from fonctions import Dataset
from src.preprocessing import display_missing_values

In [16]:
train=Dataset("data/train.csv")
data_train=train.load_data()
test=Dataset("data/test.csv")
data_test=test.load_data()

In [17]:
from fonctions import Preprocessor,TrainPreprocessor,TestPreprocessor
train_preprocessor=TrainPreprocessor(data_train)
test_preprocessor=TestPreprocessor(data_test)

In [18]:
variables_continues=["Fuel consumption ","Electric range (km)","ec (cm3)","z (Wh/km)","W (mm)"]

for col in variables_continues:
    train_preprocessor.outlier_detection(col)
    test_preprocessor.outlier_detection(col)

In [19]:
train_preprocessor.fill_fuel_consumption()
train_preprocessor.fill_electric_range()
train_preprocessor.fill_engine_capacity()
train_preprocessor.fill_electric_consumption()
train_preprocessor.fill_category_type()
train_preprocessor.fill_wheel_base()
train_preprocessor.fill_At_1()
train_preprocessor.fill_At_2()
train_preprocessor.fill_mass()
train_preprocessor.fill_engine_power()
train_preprocessor.encode_country()
train_preprocessor.encode_manufacture_pooling()
train_preprocessor.last_step()

test_preprocessor.fill_fuel_consumption()
test_preprocessor.fill_electric_range()
test_preprocessor.fill_engine_capacity()
test_preprocessor.fill_electric_consumption()
test_preprocessor.fill_category_type()
test_preprocessor.fill_wheel_base()
test_preprocessor.fill_At_1()
test_preprocessor.fill_At_2()
test_preprocessor.fill_mass()
test_preprocessor.fill_engine_power()
test_preprocessor.encode_country()
test_preprocessor.encode_manufacture_pooling()
test_preprocessor.last_step()

In [5]:
data_train = train_preprocessor.encode_that_var("Ct")
data_train = train_preprocessor.encode_that_var("Cr")
data_test = test_preprocessor.encode_that_var("Ct")
data_test = test_preprocessor.encode_that_var("Cr")

In [5]:
def create_conforme(df):
    df['conforme'] = df['Tan'].isna()
    df['conforme'] = df['conforme'].apply(lambda x: 1 if x==False else 0)
    df.drop(columns='Tan', inplace=True)
    pass
def compute_surface(obs):
    max_largeur= max(obs['At1 (mm)'], obs['At2 (mm)'])
    return obs['W (mm)']*obs['At1 (mm)'] if max_largeur == obs['At1 (mm)'] else obs['W (mm)'] * obs['At2 (mm)']

def create_surface(df):
    df['surface']= df.apply(compute_surface, axis=1)
    pass

def group_fuel_types(category: str):
    if category in ['PETROL/ELECTRIC', 'DIESEL/ELECTRIC']:
        return "HYBRID"
    elif category in ['NG-BIOMETHANE', 'HYDROGEN', 'NG','E85']:
        return "BIO-FUEL"
    elif category in ['PETROL','LPG'] :
        return 'PETROL'
    else:
        return category
def create_carburant(df):
    df['carburant']= df['Ft'].apply(group_fuel_types)
    df.drop(columns='Ft',inplace = True)

In [6]:
create_conforme(data_train)
create_surface(data_train)
create_carburant(data_train)

create_conforme(data_test)
create_surface(data_test)
create_carburant(data_test)

In [8]:
data_train = train_preprocessor.encode_that_var("carburant")
data_test = test_preprocessor.encode_that_var("carburant")

In [20]:
drop_this_cuz_no_use=['VFN', 'Mh', 'Man', 'T', 'Mk', 'Cn','Mt']
drop_this_cuz_personnal_choice = ['W (mm)', 'At1 (mm)', 'At2 (mm)','Fm','Erwltp (g/km)']

data_train.drop(columns=drop_this_cuz_no_use, inplace=True)
data_test.drop(columns=drop_this_cuz_no_use, inplace=True)

In [21]:
data_train.drop(columns='ID',inplace=True)

In [22]:
import numpy as np
import pandas as pd
import seaborn as sns
from src.preprocessing import display_missing_values
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
sns.set_theme(style="ticks", palette="pastel")

from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

# Random Forest (numerical)
No drop dupplicates, only keep numerical cols

MAE: 3.2843

In [12]:

columns_to_keep = ['m (kg)', 'Ewltp (g/km)', 'W (mm)', 'At1 (mm)', 'At2 (mm)', 'ec (cm3)', 'ep (KW)', 'z (Wh/km)', 'Fuel consumption ', 'Electric range (km)']

train, test = train_test_split(data_train[data_train.columns.intersection(columns_to_keep)], test_size=0.33, random_state=42)

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

X_train, y_train = train.drop(columns=["Ewltp (g/km)"]), train["Ewltp (g/km)"]
X_test, y_test = test.drop(columns=["Ewltp (g/km)"]), test["Ewltp (g/km)"]

In [13]:
random_forest = RandomForestRegressor(random_state=42)

random_forest.fit(X_train, y_train)

y_pred = random_forest.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 3.2811834066371595


In [16]:
data_test["Ewltp (g/km)"] = random_forest.predict(data_test[data_test.columns.intersection(columns_to_keep)])

In [17]:
data_test[["ID","Ewltp (g/km)"]].to_csv("data/reg_rf_numerical_no_drop_dups.csv", index=False)

## RF numerical (new set of predictors)

In [13]:

columns_to_keep = ['m (kg)', 'Ewltp (g/km)', 'W (mm)', 'At1 (mm)', 'At2 (mm)', 'ec (cm3)', 'ep (KW)', 'z (Wh/km)', 'Fuel consumption ']

train, test = train_test_split(data_train[data_train.columns.intersection(columns_to_keep)], test_size=0.33, random_state=42)

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

X_train, y_train = train.drop(columns=["Ewltp (g/km)"]), train["Ewltp (g/km)"]
X_test, y_test = test.drop(columns=["Ewltp (g/km)"]), test["Ewltp (g/km)"]

In [14]:
random_forest = RandomForestRegressor(random_state=42, n_jobs=-1)

random_forest.fit(X_train, y_train)

y_pred = random_forest.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 3.314147394462553


In [17]:
data_test["Ewltp (g/km)"] = random_forest.predict(data_test[data_test.columns.intersection(columns_to_keep)])

In [18]:
data_test[["ID","Ewltp (g/km)"]].to_csv("data/reg_rf_num_dups_no_e_range.csv", index=False)

## RF outliers numerical

In [16]:

columns_to_keep = ['m (kg)', 'Ewltp (g/km)', 'W (mm)', 'At1 (mm)', 'At2 (mm)', 'ec (cm3)', 'ep (KW)', 'z (Wh/km)', 'Fuel consumption ', 'Electric range (km)']

train, test = train_test_split(data_train[data_train.columns.intersection(columns_to_keep)], test_size=0.33, random_state=42)

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

X_train, y_train = train.drop(columns=["Ewltp (g/km)"]), train["Ewltp (g/km)"]
X_test, y_test = test.drop(columns=["Ewltp (g/km)"]), test["Ewltp (g/km)"]

In [18]:
random_forest = RandomForestRegressor(random_state=42, n_jobs=-1)

random_forest.fit(X_train, y_train)

y_pred = random_forest.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 3.2811834066371603


In [19]:
data_test["Ewltp (g/km)"] = random_forest.predict(data_test[data_test.columns.intersection(columns_to_keep)])

In [20]:
data_test[["ID","Ewltp (g/km)"]].to_csv("data/reg_rf_outlier.csv", index=False)

## RF outliers numerical + country

In [9]:

columns_to_keep = ['m (kg)', 'Ewltp (g/km)', 'W (mm)', 'At1 (mm)',
                    'At2 (mm)', 'ec (cm3)', 'ep (KW)', 'z (Wh/km)',
                      'Fuel consumption ', 'Electric range (km)','Country']

train, test = train_test_split(data_train[data_train.columns.intersection(columns_to_keep)], test_size=0.33, random_state=42)

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

X_train, y_train = train.drop(columns=["Ewltp (g/km)"]), train["Ewltp (g/km)"]
X_test, y_test = test.drop(columns=["Ewltp (g/km)"]), test["Ewltp (g/km)"]

In [10]:
random_forest = RandomForestRegressor(random_state=42, n_jobs=-1)

random_forest.fit(X_train, y_train)

y_pred = random_forest.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 3.185450881133795


In [13]:
data_test["Ewltp (g/km)"] = random_forest.predict(data_test[data_test.columns.intersection(columns_to_keep)])
data_test[["ID","Ewltp (g/km)"]].to_csv("data/reg_rf_country.csv", index=False)

### little test

In [9]:

columns_to_keep = ['m (kg)', 'Ewltp (g/km)', 'W (mm)', 'At1 (mm)',
                    'At2 (mm)', 'ec (cm3)', 'ep (KW)', 'z (Wh/km)',
                      'Fuel consumption ', 'Electric range (km)','Country']

train, test = train_test_split(data_train[data_train.columns.intersection(columns_to_keep)], test_size=0.33, random_state=42)

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

X_train, y_train = train.drop(columns=["Ewltp (g/km)"]), train["Ewltp (g/km)"]
X_test, y_test = test.drop(columns=["Ewltp (g/km)"]), test["Ewltp (g/km)"]

In [10]:
random_forest = RandomForestRegressor(criterion="absolute_error", random_state=42, n_jobs=-1)

random_forest.fit(X_train, y_train)

y_pred = random_forest.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

KeyboardInterrupt: 

In [ ]:
data_test["Ewltp (g/km)"] = random_forest.predict(data_test[data_test.columns.intersection(columns_to_keep)])
data_test[["ID","Ewltp (g/km)"]].to_csv("data/reg_rf_outlier_country.csv", index=False)

## RF outliers numerical + Country + Mp

In [23]:

columns_to_keep = ['m (kg)','Mp', 'Ewltp (g/km)', 
                   'W (mm)', 'At1 (mm)', 'At2 (mm)',
                     'ec (cm3)', 'ep (KW)', 'z (Wh/km)', 
                     'Fuel consumption ', 'Electric range (km)','Country']

train, test = train_test_split(data_train[data_train.columns.intersection(columns_to_keep)], test_size=0.33, random_state=42)

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

X_train, y_train = train.drop(columns=["Ewltp (g/km)"]), train["Ewltp (g/km)"]
X_test, y_test = test.drop(columns=["Ewltp (g/km)"]), test["Ewltp (g/km)"]

In [24]:
random_forest = RandomForestRegressor(random_state=42, n_jobs=-1)

random_forest.fit(X_train, y_train)

y_pred = random_forest.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 3.1826152819810107


In [25]:
data_test["Ewltp (g/km)"] = random_forest.predict(data_test[data_test.columns.intersection(columns_to_keep)])
data_test[["ID","Ewltp (g/km)"]].to_csv("data/reg_rf_outlier_country_mp.csv", index=False)

## RF (numerical) + K fold

In [21]:
columns_to_keep_no_target = ['m (kg)', 'W (mm)', 'At1 (mm)',
                    'At2 (mm)', 'ec (cm3)', 'ep (KW)', 'z (Wh/km)',
                      'Fuel consumption ', 'Electric range (km)','Country']

X_train = data_train[data_train.columns.intersection(columns_to_keep_no_target)]
y_train= data_train["Ewltp (g/km)"]

In [22]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=42, n_jobs=-1)

k_folds = 3

kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Créez des listes pour stocker les modèles et les scores
models = []
scores = []

# Effectuez la validation croisée
for train_index, test_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[test_index]

    # Entraînez le modèle sur X_train_fold et y_train_fold
    model.fit(X_train_fold, y_train_fold)

    # Évaluez le modèle sur X_val_fold (vous pouvez utiliser la métrique de votre choix, par exemple MAE)
    y_pred = model.predict(X_val_fold)
    mae = mean_absolute_error(y_val_fold, y_pred)
    
    # Stockez le modèle et le score
    models.append(model)
    scores.append(mae)

# Affichez les scores MAE pour chaque pli
for i, mae in enumerate(scores):
    print(f"Fold {i + 1} MAE: {mae}")

# Calculez la moyenne des scores MAE
average_mae = sum(scores) / k_folds
print(f"Average MAE: {average_mae}")

best_model_index = scores.index(min(scores))
best_model = models[best_model_index]

print(f"min MAE: {min(scores)}")

KeyboardInterrupt: 

In [ ]:
data_test["Ewltp (g/km)"] = best_model.predict(data_test.drop(columns='ID'))

In [ ]:
data_test[["ID","Ewltp (g/km)"]].to_csv("data/reg_rf_numerical_kf.csv", index=False)

## RF validation set 

In [26]:
columns_to_keep = ['m (kg)','Mp', 'Ewltp (g/km)', 
                   'W (mm)', 'At1 (mm)', 'At2 (mm)',
                     'ec (cm3)', 'ep (KW)', 'z (Wh/km)', 
                     'Fuel consumption ', 'Electric range (km)','Country']
train_data, rest_data = train_test_split(data_train[data_train.columns.intersection(columns_to_keep)], test_size=0.33, random_state=42)

validation_data, test_data = train_test_split(rest_data, test_size=0.5, random_state=42)

# Vérifiez les tailles des ensembles
print("Taille de l'ensemble d'entraînement :", len(train_data))
print("Taille de l'ensemble de validation :", len(validation_data))
print("Taille de l'ensemble de test :", len(test_data))


Taille de l'ensemble d'entraînement : 5073004
Taille de l'ensemble de validation : 1249322
Taille de l'ensemble de test : 1249323


In [27]:

# Divisez les ensembles en caractéristiques (X) et cible (y)
X_train, y_train = train_data.drop(columns=["Ewltp (g/km)"]), train_data["Ewltp (g/km)"]
X_val, y_val = validation_data.drop(columns=["Ewltp (g/km)"]), validation_data["Ewltp (g/km)"]
X_test, y_test = test_data.drop(columns=["Ewltp (g/km)"]), test_data["Ewltp (g/km)"]

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 7, 10],
}

# Créez votre modèle
base_model = RandomForestRegressor(random_state=42, n_jobs=-1)

# Utilisez GridSearchCV pour la recherche sur grille avec validation croisée
grid_search = GridSearchCV(estimator=base_model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=3)
grid_search.fit(X_val, y_val)

# Affichez les meilleurs hyperparamètres trouvés
best_params = grid_search.best_params_
print("Meilleurs hyperparamètres :", best_params)

# Utilisez le meilleur modèle trouvé sur l'ensemble de test
best_model = grid_search.best_estimator_
y_test_pred = best_model.predict(X_test)
mae_test = mean_absolute_error(y_test, y_test_pred)
print("MAE sur l'ensemble de test avec le meilleur modèle :", mae_test)



Meilleurs hyperparamètres : {'max_depth': 10, 'n_estimators': 150}
MAE sur l'ensemble de test avec le meilleur modèle : 10.80078252858729


## RF with test_fonctions script

### Import

In [1]:
from test_fonctions import Dataset
from src.preprocessing import display_missing_values

In [2]:
train=Dataset("data/train.csv")
data_train=train.load_data()
test=Dataset("data/test.csv")
data_test=test.load_data()

In [3]:
from test_fonctions import Preprocessor,TrainPreprocessor,TestPreprocessor
train_preprocessor=TrainPreprocessor(data_train)
test_preprocessor=TestPreprocessor(data_test)

In [4]:
variables_continues=["Fuel consumption ","Electric range (km)","ec (cm3)","z (Wh/km)","W (mm)","Mt"]

for col in variables_continues:
    train_preprocessor.outlier_detection(col)
    test_preprocessor.outlier_detection(col)

In [5]:
train_preprocessor.fill_fuel_consumption()
train_preprocessor.fill_electric_range()
train_preprocessor.fill_engine_capacity()
train_preprocessor.fill_electric_consumption()
train_preprocessor.fill_category_type()
train_preprocessor.fill_wheel_base()
train_preprocessor.fill_At_1()
train_preprocessor.fill_At_2()
train_preprocessor.fill_mass()
train_preprocessor.fill_test_mass()
train_preprocessor.fill_fuel_mode()
train_preprocessor.fill_engine_power()
train_preprocessor.encode_country()
train_preprocessor.encode_manufacture_pooling()
train_preprocessor.encode_fuel_mode()
train_preprocessor.encode_fuel_type()
train_preprocessor.last_step()

test_preprocessor.fill_fuel_consumption()
test_preprocessor.fill_electric_range()
test_preprocessor.fill_engine_capacity()
test_preprocessor.fill_electric_consumption()
test_preprocessor.fill_category_type()
test_preprocessor.fill_wheel_base()
test_preprocessor.fill_At_1()
test_preprocessor.fill_At_2()
test_preprocessor.fill_mass()
test_preprocessor.fill_test_mass()
test_preprocessor.fill_fuel_mode()
test_preprocessor.fill_engine_power()
test_preprocessor.encode_country()
test_preprocessor.encode_manufacture_pooling()
test_preprocessor.encode_fuel_mode()
test_preprocessor.encode_fuel_type()
test_preprocessor.last_step()

In [6]:
def create_conforme(df):
    df['conforme'] = df['Tan'].isna()
    df['conforme'] = df['conforme'].apply(lambda x: 1 if x==False else 0)
    df.drop(columns='Tan', inplace=True)
    pass
def compute_surface(obs):
    max_largeur= max(obs['At1 (mm)'], obs['At2 (mm)'])
    return obs['W (mm)']*obs['At1 (mm)'] if max_largeur == obs['At1 (mm)'] else obs['W (mm)'] * obs['At2 (mm)']

def create_surface(df):
    df['surface']= df.apply(compute_surface, axis=1)
    pass

create_conforme(data_train)
create_surface(data_train)

create_conforme(data_test)
create_surface(data_test)

In [7]:
drop_this_cuz_no_use=['VFN', 'Mh', 'Man', 'T', 'Mk', 'Cn','Mt']
# drop_this_cuz_personnal_choice = ['W (mm)', 'At1 (mm)', 'At2 (mm)','Fm','Erwltp (g/km)']

data_train.drop(columns=drop_this_cuz_no_use, inplace=True)
data_test.drop(columns=drop_this_cuz_no_use, inplace=True)

data_train.drop(columns='ID',inplace=True)

In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
from src.preprocessing import display_missing_values
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
sns.set_theme(style="ticks", palette="pastel")

from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

### Model testing

In [9]:
data_train.drop(columns=['Ct','Cr','Erwltp (g/km)'], inplace=True)
data_test.drop(columns=['Ct','Cr','Erwltp (g/km)'], inplace=True)

In [13]:
train, test = train_test_split(data_train, test_size=0.33, random_state=42)

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

X_train, y_train = train.drop(columns=["Ewltp (g/km)"]), train["Ewltp (g/km)"]
X_test, y_test = test.drop(columns=["Ewltp (g/km)"]), test["Ewltp (g/km)"]

In [14]:
random_forest = RandomForestRegressor(random_state=42, n_jobs=-1)

random_forest.fit(X_train, y_train)

y_pred = random_forest.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 3.1738805484592563


In [16]:
data_test["Ewltp (g/km)"] = random_forest.predict(data_test.drop(columns='ID'))
data_test[["ID","Ewltp (g/km)"]].to_csv("data/rf_num_and_cat.csv", index=False)

In [17]:
data_train.head()

,Country,Mp,m (kg),Ewltp (g/km),W (mm),At1 (mm),At2 (mm),Ft,Fm,ec (cm3),ep (KW),z (Wh/km),Fuel consumption,Electric range (km),flag_Fuel consumption,flag_Electric range (km),flag_ec (cm3),flag_z (Wh/km),flag_W (mm),flag_Mt,conforme,surface
0,5,1,1387.00,401.05,2700.00,1571.00,1576.00,8.00,3.00,999.00,92.00,0.00,5.60,0.00,0,0,0,0,0,0,1,4255200.00
1,10,10,1172.00,394.68,2552.00,1500.00,1483.00,8.00,4.00,999.00,70.00,0.00,5.50,0.00,0,0,0,0,0,0,1,3828000.00
2,10,10,1204.00,398.56,2552.00,1500.00,1483.00,8.00,4.00,999.00,70.00,0.00,5.60,0.00,0,0,0,0,0,0,1,3828000.00
3,5,3,1438.00,479.47,2650.00,1555.00,1563.00,8.00,4.00,1591.00,150.00,0.00,6.80,0.00,0,0,0,0,0,0,1,4141950.00
4,10,10,1207.00,421.85,2552.00,1500.00,1483.00,8.00,4.00,999.00,81.00,0.00,5.90,0.00,0,0,0,0,0,0,1,3828000.00


In [ ]:
train, test = train_test_split(data_train,test_size=0.33,random_state=42)

train.reset_index(drop=True, inplace=True ) #car ça fout la merde dans l'index
test.reset_index(drop = True, inplace = True)

X_train, y_train =train.drop(columns=["Ewltp (g/km)"]), train["Ewltp (g/km)"]
X_test, y_test =test.drop(columns=["Ewltp (g/km)"]), test["Ewltp (g/km)"]

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

# Créez un modèle XGBoost
xgb_model = XGBRegressor(objective="reg:squarederror", random_state=42)

# Liste des hyperparamètres à régler et leurs plages de valeurs
param_dist = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 2, 3],
    # Ajoutez d'autres hyperparamètres ici
}

# Recherche aléatoire des hyperparamètres
random_search = RandomizedSearchCV(xgb_model, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=-1, random_state=42)

# Exécutez la recherche aléatoire sur les données d'entraînement
random_search.fit(X_train, y_train)

# Obtenez les meilleurs hyperparamètres
best_params = random_search.best_params_
print("Meilleurs hyperparamètres :", best_params)

# Utilisez les meilleurs hyperparamètres pour entraîner le modèle final
best_xgb_model = XGBRegressor(objective="reg:squarederror", random_state=42, **best_params)
best_xgb_model.fit(X_train, y_train)

y_pred =best_xgb_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)